# Train CNNs and obtain classification accuracies

# Image classifier (H-CNN)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


data_dir = 'E:Grass_Images_Split_01'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train Models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from PIL import Image
import os
import numpy as np
import copy
import random
import math

SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Ensure deterministic behavior
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation((-90, 90)),
        transforms.ToTensor(),
    ]),
    'train_strong': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation((-90, 90)),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.RandomAutocontrast(p=0.5),
        transforms.RandomEqualize(p=0.5),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}


labeled_data_dir = "E:Grass_Images_Split_01/"
unlabeled_data_dir = "E:Grass_Images_Fossils_And_Subfossils/"

class UnlabeledDataset(Dataset):
    def __init__(self, file_paths, transform=None, strong_transform=None):
        self.file_paths = file_paths
        self.transform = transform
        self.strong_transform = strong_transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        weak_image = self.transform(image) if self.transform else image
        strong_image = self.strong_transform(image) if self.strong_transform else image
        return weak_image, strong_image

labeled_dataset = datasets.ImageFolder(os.path.join(labeled_data_dir, 'train'), data_transforms['train'])
labeled_dataset_size = len(labeled_dataset)

unlabeled_file_paths = [os.path.join(unlabeled_data_dir, img) for img in os.listdir(unlabeled_data_dir)]
unlabeled_dataset = UnlabeledDataset(unlabeled_file_paths, data_transforms['train'], data_transforms['train_strong'])
unlabeled_dataset_size = len(unlabeled_dataset)

mu = math.ceil(unlabeled_dataset_size / labeled_dataset_size)
labeled_batch_size = 10
unlabeled_batch_size = int(labeled_batch_size * mu)

labeled_loader = DataLoader(labeled_dataset, batch_size=labeled_batch_size, shuffle=True, num_workers=0)
val_dataset = datasets.ImageFolder(os.path.join(labeled_data_dir, 'val'), data_transforms['val'])
val_loader = DataLoader(val_dataset, batch_size=labeled_batch_size, shuffle=False, num_workers=0)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=unlabeled_batch_size, shuffle=True, num_workers=0)

model = models.resnext101_32x8d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(labeled_dataset.classes))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

lambda_u = 1
T = 1
threshold = 0.95
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0009, momentum=0.9)
step_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

def sharpen(probabilities, temperature):
    res = torch.pow(probabilities, 1.0 / temperature)
    return torch.div(res, torch.sum(res, dim=1, keepdim=True))

num_epochs = 30
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        if phase == 'train':
            iter_unlabeled = iter(unlabeled_loader)
            for inputs_l, labels in labeled_loader:
                inputs_l, labels = inputs_l.to(device), labels.to(device)
                try:
                    inputs_u_w, inputs_u_s = next(iter_unlabeled)
                except StopIteration:
                    iter_unlabeled = iter(unlabeled_loader)
                    inputs_u_w, inputs_u_s = next(iter_unlabeled)
                inputs_u_w, inputs_u_s = inputs_u_w.to(device), inputs_u_s.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs_l = model(inputs_l)
                    loss_l = criterion(outputs_l, labels).mean()

                    with torch.no_grad():
                        outputs_u_w = model(inputs_u_w)
                        pseudo_labels = torch.softmax(outputs_u_w, dim=1)
                        pseudo_labels = sharpen(pseudo_labels, T)
                        max_probs, pseudo_labels = torch.max(pseudo_labels, dim=1)
                        mask = max_probs.ge(threshold).float()
                        pseudo_labels = pseudo_labels[mask > 0]
                        inputs_u_s = inputs_u_s[mask > 0]

                    if len(inputs_u_s) > 0:
                        outputs_u_s = model(inputs_u_s)
                        loss_u = criterion(outputs_u_s, pseudo_labels)
                        loss_u = (lambda_u * loss_u).mean()
                    else:
                        loss_u = 0

                    loss = loss_l + loss_u
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs_l.size(0)
                _, preds = torch.max(outputs_l, 1)
                running_corrects += torch.sum(preds == labels.data)

            step_lr_scheduler.step()

        else:
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels).mean()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            epoch_loss = running_loss / labeled_dataset_size
            epoch_acc = running_corrects.double() / labeled_dataset_size
        elif phase == 'val':
            epoch_loss = running_loss / len(val_dataset)
            epoch_acc = running_corrects.double() / len(val_dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

print('Best val Acc: {:4f}'.format(best_acc))
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), 'Semi_Supervised_Grass_Images_Split01.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from PIL import Image
import os
import numpy as np
import copy
import random
import math

SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation((-90, 90)),
        transforms.ToTensor(),
    ]),
    'train_strong': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation((-90, 90)),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.RandomAutocontrast(p=0.5),
        transforms.RandomEqualize(p=0.5),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

labeled_data_dir = "E:Grass_Patches_Split_01/"
unlabeled_data_dir = "E:Grass_Patches_Fossils_And_Subfossils/"

def collect_image_paths(directory):
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                image_paths.append(os.path.join(root, file))
    return image_paths

class UnlabeledDataset(Dataset):
    def __init__(self, file_paths, transform=None, strong_transform=None):
        self.file_paths = file_paths
        self.transform = transform
        self.strong_transform = strong_transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except (PermissionError, FileNotFoundError) as e:
            print(f"Error opening image {img_path}: {e}")
            return None, None

        weak_image = self.transform(image) if self.transform else image
        strong_image = self.strong_transform(image) if self.strong_transform else image
        return weak_image, strong_image

labeled_dataset = datasets.ImageFolder(os.path.join(labeled_data_dir, 'train'), data_transforms['train'])
labeled_dataset_size = len(labeled_dataset)

unlabeled_file_paths = collect_image_paths(unlabeled_data_dir)
unlabeled_dataset = UnlabeledDataset(unlabeled_file_paths, data_transforms['train'], data_transforms['train_strong'])
unlabeled_dataset_size = len(unlabeled_dataset)

mu = math.ceil(unlabeled_dataset_size / labeled_dataset_size)
labeled_batch_size = 10
unlabeled_batch_size = int(labeled_batch_size * mu)

labeled_loader = DataLoader(labeled_dataset, batch_size=labeled_batch_size, shuffle=True, num_workers=0)
val_dataset = datasets.ImageFolder(os.path.join(labeled_data_dir, 'val'), data_transforms['val'])
val_loader = DataLoader(val_dataset, batch_size=labeled_batch_size, shuffle=False, num_workers=0)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=unlabeled_batch_size, shuffle=True, num_workers=0)

model = models.resnext101_32x8d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(labeled_dataset.classes))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

lambda_u = 1
T = 1
threshold = 0.95
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0009, momentum=0.9)
step_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

def sharpen(probabilities, temperature):
    res = torch.pow(probabilities, 1.0 / temperature)
    return torch.div(res, torch.sum(res, dim=1, keepdim=True))

num_epochs = 30
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        if phase == 'train':
            iter_unlabeled = iter(unlabeled_loader)
            for inputs_l, labels in labeled_loader:
                inputs_l, labels = inputs_l.to(device), labels.to(device)
                try:
                    inputs_u_w, inputs_u_s = next(iter_unlabeled)
                    if inputs_u_w is None or inputs_u_s is None:
                        continue
                except StopIteration:
                    iter_unlabeled = iter(unlabeled_loader)
                    inputs_u_w, inputs_u_s = next(iter_unlabeled)
                    if inputs_u_w is None or inputs_u_s is None:
                        continue
                inputs_u_w, inputs_u_s = inputs_u_w.to(device), inputs_u_s.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs_l = model(inputs_l)
                    loss_l = criterion(outputs_l, labels).mean()

                    with torch.no_grad():
                        outputs_u_w = model(inputs_u_w)
                        pseudo_labels = torch.softmax(outputs_u_w, dim=1)
                        pseudo_labels = sharpen(pseudo_labels, T)
                        max_probs, pseudo_labels = torch.max(pseudo_labels, dim=1)
                        mask = max_probs.ge(threshold).float()
                        pseudo_labels = pseudo_labels[mask > 0]
                        inputs_u_s = inputs_u_s[mask > 0]

                    if len(inputs_u_s) > 0:
                        outputs_u_s = model(inputs_u_s)
                        loss_u = criterion(outputs_u_s, pseudo_labels)
                        loss_u = (lambda_u * loss_u).mean()
                    else:
                        loss_u = 0

                    loss = loss_l + loss_u
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs_l.size(0)
                _, preds = torch.max(outputs_l, 1)
                running_corrects += torch.sum(preds == labels.data)

            step_lr_scheduler.step()

        else:
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels).mean()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            epoch_loss = running_loss / labeled_dataset_size
            epoch_acc = running_corrects.double() / labeled_dataset_size
        elif phase == 'val':
            epoch_loss = running_loss / len(val_dataset)
            epoch_acc = running_corrects.double() / len(val_dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

print('Best val Acc: {:4f}'.format(best_acc))
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), 'Semi_Supervised_Grass_Patches_Split01.pth')

# Calculate Classification Accuracies (H-CNN)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

# Forward Pass Images to the network to get (1) Softmax probabilities and (2) best classification prediction

PATH = "Semi_Supervised_Grass_Images_Split01.pth"

model =  models.resnext101_32x8d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names));
model.load_state_dict(torch.load(PATH));
#model.load_state_dict(torch.load(PATH, map_location="cpu"))
model.to(device);

import glob
from PIL import Image

transform = transforms.Compose([
    torchvision.transforms.Resize((224,224)),
        transforms.ToTensor(),
])

def predict(model, image_dir):
    """
    """
    patch_paths = glob.glob(os.path.join(image_dir, '*'))
    images = []
    for path in patch_paths:
        image = Image.open(path).convert('RGB')
        image = transform(image)
        images.append(image)

    images = torch.stack(images, 0).to(device)
    logits = model(images)
    #probs = torch.softmax(logits, 1).detach().cpu().numpy()
    probs = logits.detach().cpu().numpy()
    probs = torch.tensor(probs)
    probs = probs.mean(0)
    probs = torch.softmax(probs, 0).detach().cpu().numpy()
    # Take average and get final prediction
    # avg_probs = probs.mean(0)
    avg_probs = probs
    final_pred = avg_probs.argmax()
    return final_pred, avg_probs

# Validation dir

val_dir = 'E:Grass_Images_Split_01/val'

val_class_dirs = glob.glob(os.path.join(val_dir, '*'))
model.eval()
class_map = {name: idx for idx, name in enumerate(class_names)}
results = []
all_preds = []
labels = []
all_image_dirs = []

for d in val_class_dirs:
    image_dirs = glob.glob(os.path.join(d, '*'))
    for image_dir in image_dirs:
        class_name = os.path.basename(image_dir).split('.')[0]
        label = class_map[class_name]
        labels.append(label)
        all_image_dirs.append(image_dir)
        class_id, probs = predict(model, image_dir)
        all_preds.append(probs)

        results.append((image_dir, class_id))
        print(f'Predict {image_dir} as class {class_id}')

all_preds = np.stack(all_preds, 0)
labels = np.array(labels)
print(all_preds.shape, all_preds.dtype, labels.shape, labels.dtype)
print('Accuracy: {}'.format((all_preds.argmax(1) == labels).mean()))

# Plot Confusion Matrix (H-CNN)



In [ ]:
import os

# Path to the main folder
main_folder_path = "E:Grass_Images_Split_01/"

# Get a list of all subfolders in the main folder
subfolders = [f.path for f in os.scandir(main_folder_path) if f.is_dir()]

# Dictionary to store the number of folders in each subfolder
subfolder_counts = {}

# Iterate through each subfolder and count the number of subfolders they contain
for subfolder in subfolders:
    subfolder_name = os.path.basename(subfolder)
    inner_folders = [f.path for f in os.scandir(subfolder) if f.is_dir()]
    subfolder_counts[subfolder_name] = len(inner_folders)

# Print the results
for subfolder_name, count in subfolder_counts.items():
    print(f"Subfolder: {subfolder_name}, Number of inner folders: {count}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

updated_taxon_names = ["Elymus ciliaris","Eragrostis mexicana","Agrostis quinqueseta","Agrostis trachyphylla","Agrostis volkensii","Amphibromus neesii","Andropogon amethystinus","Andropogon chrysostachyus","Heteropogon contortus","Andropogon lima","Andropogon schirensis","Anthoxanthum nivale","Aristida megapotamica","Bothriochloa bladhii","Urochloa brizantha","Brachypodium flexum","Bromus auleticus","Bromus orcuttianus","Bromus ciliatus","Bromus lanatus","Bromus leptoclados","Bromus hordeaceus subsp. thominei","Calamagrostis epigejos","Chrysopogon fallax","Cymbopogon nardus","Cynodon dactylon","Dactylis glomerata","Digitaria abyssinica","Echinopogon caespitosus","Ehrharta erecta","Ehrharta erecta var. natalensis","Eleusine coracana","Eleusine jaegeri","Exotheca abyssinica","Pseudobromus africanus","Festuca costata","Ampelodesmos mauritanicus","Isachne mauritiana","Koeleria capensis","Lolium perenne","Melica onoei","Miscanthidium violaceum","Oplismenus compositus","Oplismenus hirtellus","Cenchrus clandestinus","Pentameris borussica","Phalaris arundinacea","Poa anceps","Poa leptoclada","Poa schimperiana","Tripidium arundinaceum","Secale cereale","Setaria megaphylla","Oldeania alpina","Sorghum halepense","Sporobolus michauxianus","Austrostipa flavescens","Koordersiochloa longiarista","Themeda triandra","Zea mays"];

plt.rcParams['font.family'] = 'Arial'

TaxonName = updated_taxon_names

# Compute confusion matrix
cm = confusion_matrix(labels, all_preds.argmax(1))

# Sort TaxonName alphabetically and get sorted indices
sorted_indices = np.argsort(TaxonName)
sorted_TaxonName = np.array(TaxonName)[sorted_indices]

# Reorder the confusion matrix rows and columns based on sorted indices
cm_sorted = cm[sorted_indices, :][:, sorted_indices]

# Normalize confusion matrix by the sum of rows
cm_normalized = cm_sorted.astype('float') / cm_sorted.sum(axis=1)[:, np.newaxis]

# Ensure no division by zero
cm_normalized[np.isnan(cm_normalized)] = 0

# Compute confusion matrix for calculating accuracies
cm_full = confusion_matrix(labels, all_preds.argmax(1))

# Calculate the accuracy for each taxon
accuracies = np.diag(cm_full) / cm_full.sum(axis=1) * 100

# Reorder accuracies to match the sorted taxon names
reordered_accuracies = accuracies[sorted_indices]

# Function to italicize taxon names except "var." and "subsp."
def italicize_name(name):
    words = name.split()
    italicized_words = [f"$\\it{{{word}}}$" if word not in ["var.", "subsp."] else word for word in words]
    return " ".join(italicized_words)

# Get the specimen counts sorted according to sorted_TaxonName
sorted_specimen_counts = [subfolder_counts[name] for name in sorted_TaxonName]

# Update taxon names with accuracy in parentheses for x-axis
updated_taxon_names_with_accuracy = [
    f"{italicize_name(name)} ({reordered_accuracies[idx]:.0f})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Update taxon names with specimen count in parentheses for y-axis
updated_taxon_names_with_counts = [
    f"{italicize_name(name)} ({sorted_specimen_counts[idx]})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Plot the confusion matrix
plt.figure(figsize=(35, 27), dpi=300)
ax = sns.heatmap(cm_normalized, annot=False, fmt=".2f", vmax=1, xticklabels=updated_taxon_names_with_accuracy, yticklabels=updated_taxon_names_with_counts, cmap='cividis', linewidths=.0)
ax.set_xlabel('Predicted Taxon (%)', fontsize=32)
ax.set_ylabel('True Taxon (N)', fontsize=32)

# Adjust tick parameters
plt.tick_params(axis='x', which='major', labelsize=30)
plt.tick_params(axis='y', which='major', labelsize=30)

# Modify colorbar font size
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30)

# Italicize the tick labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', rotation_mode='anchor', fontsize=24)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=24)

ax.xaxis.set_tick_params(pad=10)

plt.show()

In [ ]:
cm_images_01 = cm_normalized

In [ ]:
# Concatenate Image Directories and Softmax Probabilities, and get sorted probability vectors

import numpy as np
Dirs_and_Scores = {}
for i in range (len(all_image_dirs)):
  array_dir = all_image_dirs[i]
  array_pred = all_preds[i]
  Dirs_and_Scores[array_dir] = array_pred
  #Dirs_and_Scores.append(Dir_and_Score)

Image_Probs_Sorted = []
for key in sorted(Dirs_and_Scores.keys()) :
   #print(key , " :: " , Dirs_and_Scores[key])
   #Dirs_and_Scores[key]
   Image_Probs_Sorted.append(Dirs_and_Scores[key])

# Image (Whole Images) Labels
labels.sort()
ImageLabels = labels

# Calculate Classification Accuracies (P-CNN)

In [ ]:
import os

# Path to the main folder
main_folder_path = "E:Grass_Patches_Split_01/"

# Get a list of all subfolders in the main folder
subfolders = [f.path for f in os.scandir(main_folder_path) if f.is_dir()]

# Dictionary to store the number of folders in each subfolder
subfolder_counts = {}

# Iterate through each subfolder and count the number of subfolders they contain
for subfolder in subfolders:
    subfolder_name = os.path.basename(subfolder)
    inner_folders = [f.path for f in os.scandir(subfolder) if f.is_dir()]
    subfolder_counts[subfolder_name] = len(inner_folders)

# Print the results
for subfolder_name, count in subfolder_counts.items():
    print(f"Subfolder: {subfolder_name}, Number of inner folders: {count}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


PATH = "Semi_Supervised_Grass_Patches_Split01.pth"

model =  models.resnext101_32x8d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names));
model.load_state_dict(torch.load(PATH));
#model.load_state_dict(torch.load(PATH, map_location="cpu"))
model.to(device);

import glob
from PIL import Image

transform = transforms.Compose([
    torchvision.transforms.Resize((224,224)),
        transforms.ToTensor(),
])

def predict(model, image_dir):
    """
    """
    patch_paths = glob.glob(os.path.join(image_dir, '*'))
    images = []
    for path in patch_paths:
        image = Image.open(path).convert('RGB')
        image = transform(image)
        images.append(image)

    images = torch.stack(images, 0).to(device)
    logits = model(images)
    #probs = torch.softmax(logits, 1).detach().cpu().numpy()
    probs = logits.detach().cpu().numpy()
    probs = torch.tensor(probs)
    probs = probs.mean(0)
    probs = torch.softmax(probs, 0).detach().cpu().numpy()
    # Take average and get final prediction
    # avg_probs = probs.mean(0)
    avg_probs = probs
    final_pred = avg_probs.argmax()
    return final_pred, avg_probs

# Validation dir

val_dir = 'E:Grass_Patches_Split_01/val'

val_class_dirs = glob.glob(os.path.join(val_dir, '*'))
model.eval()
class_map = {name: idx for idx, name in enumerate(class_names)}
results = []
all_preds = []
labels = []
all_image_dirs = []

for d in val_class_dirs:
    image_dirs = glob.glob(os.path.join(d, '*'))
    for image_dir in image_dirs:
        class_name = os.path.basename(image_dir).split('.')[0]
        label = class_map[class_name]
        labels.append(label)
        all_image_dirs.append(image_dir)
        class_id, probs = predict(model, image_dir)
        all_preds.append(probs)

        results.append((image_dir, class_id))
        print(f'Predict {image_dir} as class {class_id}')

all_preds = np.stack(all_preds, 0)
labels = np.array(labels)
print(all_preds.shape, all_preds.dtype, labels.shape, labels.dtype)
print('Accuracy: {}'.format((all_preds.argmax(1) == labels).mean()))

# Plot Confusion Matrix (P-CNN)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

updated_taxon_names = ["Elymus ciliaris","Eragrostis mexicana","Agrostis quinqueseta","Agrostis trachyphylla","Agrostis volkensii","Amphibromus neesii","Andropogon amethystinus","Andropogon chrysostachyus","Heteropogon contortus","Andropogon lima","Andropogon schirensis","Anthoxanthum nivale","Aristida megapotamica","Bothriochloa bladhii","Urochloa brizantha","Brachypodium flexum","Bromus auleticus","Bromus orcuttianus","Bromus ciliatus","Bromus lanatus","Bromus leptoclados","Bromus hordeaceus subsp. thominei","Calamagrostis epigejos","Chrysopogon fallax","Cymbopogon nardus","Cynodon dactylon","Dactylis glomerata","Digitaria abyssinica","Echinopogon caespitosus","Ehrharta erecta","Ehrharta erecta var. natalensis","Eleusine coracana","Eleusine jaegeri","Exotheca abyssinica","Pseudobromus africanus","Festuca costata","Ampelodesmos mauritanicus","Isachne mauritiana","Koeleria capensis","Lolium perenne","Melica onoei","Miscanthidium violaceum","Oplismenus compositus","Oplismenus hirtellus","Cenchrus clandestinus","Pentameris borussica","Phalaris arundinacea","Poa anceps","Poa leptoclada","Poa schimperiana","Tripidium arundinaceum","Secale cereale","Setaria megaphylla","Oldeania alpina","Sorghum halepense","Sporobolus michauxianus","Austrostipa flavescens","Koordersiochloa longiarista","Themeda triandra","Zea mays"];

plt.rcParams['font.family'] = 'Arial'

TaxonName = updated_taxon_names

# Compute confusion matrix
cm = confusion_matrix(labels, all_preds.argmax(1))

# Sort TaxonName alphabetically and get sorted indices
sorted_indices = np.argsort(TaxonName)
sorted_TaxonName = np.array(TaxonName)[sorted_indices]

# Reorder the confusion matrix rows and columns based on sorted indices
cm_sorted = cm[sorted_indices, :][:, sorted_indices]

# Normalize confusion matrix by the sum of rows
cm_normalized = cm_sorted.astype('float') / cm_sorted.sum(axis=1)[:, np.newaxis]

# Ensure no division by zero
cm_normalized[np.isnan(cm_normalized)] = 0

# Compute confusion matrix for calculating accuracies
cm_full = confusion_matrix(labels, all_preds.argmax(1))

# Calculate the accuracy for each taxon
accuracies = np.diag(cm_full) / cm_full.sum(axis=1) * 100

# Reorder accuracies to match the sorted taxon names
reordered_accuracies = accuracies[sorted_indices]

# Function to italicize taxon names except "var." and "subsp."
def italicize_name(name):
    words = name.split()
    italicized_words = [f"$\\it{{{word}}}$" if word not in ["var.", "subsp."] else word for word in words]
    return " ".join(italicized_words)

# Get the specimen counts sorted according to sorted_TaxonName
sorted_specimen_counts = [subfolder_counts[name] for name in sorted_TaxonName]

# Update taxon names with accuracy in parentheses for x-axis
updated_taxon_names_with_accuracy = [
    f"{italicize_name(name)} ({reordered_accuracies[idx]:.0f})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Update taxon names with specimen count in parentheses for y-axis
updated_taxon_names_with_counts = [
    f"{italicize_name(name)} ({sorted_specimen_counts[idx]})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Plot the confusion matrix
plt.figure(figsize=(35, 27), dpi=300)
ax = sns.heatmap(cm_normalized, annot=False, fmt=".2f", vmax=1, xticklabels=updated_taxon_names_with_accuracy, yticklabels=updated_taxon_names_with_counts, cmap='cividis', linewidths=.0)
ax.set_xlabel('Predicted Taxon (%)', fontsize=32)
ax.set_ylabel('True Taxon (N)', fontsize=32)

# Adjust tick parameters
plt.tick_params(axis='x', which='major', labelsize=30)
plt.tick_params(axis='y', which='major', labelsize=30)

# Modify colorbar font size
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30)

# Italicize the tick labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', rotation_mode='anchor', fontsize=24)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=24)

ax.xaxis.set_tick_params(pad=10)

plt.show()

In [ ]:
cm_patches_01 = cm_normalized

In [ ]:
# Concatenate Image Directories and Softmax Probabilities, and get Sorted probability vectors

import numpy as np
Dirs_and_Scores = {}
for i in range (len(all_image_dirs)):
  array_dir = all_image_dirs[i]
  array_pred = all_preds[i]
  Dirs_and_Scores[array_dir] = array_pred
  #Dirs_and_Scores.append(Dir_and_Score)

Patch_Probs_Sorted = []
for key in sorted(Dirs_and_Scores.keys()) :
   #print(key , " :: " , Dirs_and_Scores[key])
   #Dirs_and_Scores[key]
   Patch_Probs_Sorted.append(Dirs_and_Scores[key])

# Patch Labels
labels.sort()
PatchLabels = labels

# Calculate fused classification accuracy (combining H-CNN and P-CNN)

In [ ]:
import torch
import torch.nn.functional as f

# Fuse softmax score vectors across the two modalities (image and patch CNNs):

Fused_Probabilities = torch.tensor(Image_Probs_Sorted) * torch.tensor(Patch_Probs_Sorted)

# Normalize to [0-1] probability vectors

#Fused_Probabilities_Normalized = f.normalize(torch.tensor(Fused_Probabilities), p=2, dim=1)
Fused_Probabilities_Normalized = Fused_Probabilities / Fused_Probabilities.sum(dim=1, keepdim=True)


# Save Fused Probabilities as File
#np.savetxt(absolute_path + "/Fused_Probabilities_Normalized.csv", Fused_Probabilities_Normalized, delimiter=",")

#print(Fused_Probabilities_Normalized.shape)
Fused_Probabilities_Normalized_array = np.array(Fused_Probabilities_Normalized)

# Compute Fused Classification Accuracy
Indices = []
for i in range(len(Fused_Probabilities_Normalized_array)):
  index, = np.where(Fused_Probabilities_Normalized_array[i] == Fused_Probabilities_Normalized_array[i].max())
  Indices.append(index.item())

Accuracy = []
for i in range(len(Indices)):
   Per_Instance_Accuracy = Indices[i] == ImageLabels[i]
   Accuracy.append(Per_Instance_Accuracy)

Fused_Classification_Accuracy = sum(Accuracy)/len(Fused_Probabilities_Normalized_array)
print('Fused Classification Accuracy: {}'.format(Fused_Classification_Accuracy))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

updated_taxon_names = ["Elymus ciliaris","Eragrostis mexicana","Agrostis quinqueseta","Agrostis trachyphylla","Agrostis volkensii","Amphibromus neesii","Andropogon amethystinus","Andropogon chrysostachyus","Heteropogon contortus","Andropogon lima","Andropogon schirensis","Anthoxanthum nivale","Aristida megapotamica","Bothriochloa bladhii","Urochloa brizantha","Brachypodium flexum","Bromus auleticus","Bromus orcuttianus","Bromus ciliatus","Bromus lanatus","Bromus leptoclados","Bromus hordeaceus subsp. thominei","Calamagrostis epigejos","Chrysopogon fallax","Cymbopogon nardus","Cynodon dactylon","Dactylis glomerata","Digitaria abyssinica","Echinopogon caespitosus","Ehrharta erecta","Ehrharta erecta var. natalensis","Eleusine coracana","Eleusine jaegeri","Exotheca abyssinica","Pseudobromus africanus","Festuca costata","Ampelodesmos mauritanicus","Isachne mauritiana","Koeleria capensis","Lolium perenne","Melica onoei","Miscanthidium violaceum","Oplismenus compositus","Oplismenus hirtellus","Cenchrus clandestinus","Pentameris borussica","Phalaris arundinacea","Poa anceps","Poa leptoclada","Poa schimperiana","Tripidium arundinaceum","Secale cereale","Setaria megaphylla","Oldeania alpina","Sorghum halepense","Sporobolus michauxianus","Austrostipa flavescens","Koordersiochloa longiarista","Themeda triandra","Zea mays"];

plt.rcParams['font.family'] = 'Arial'

TaxonName = updated_taxon_names

# Compute confusion matrix
cm = confusion_matrix(labels, Indices)

# Sort TaxonName alphabetically and get sorted indices
sorted_indices = np.argsort(TaxonName)
sorted_TaxonName = np.array(TaxonName)[sorted_indices]

# Reorder the confusion matrix rows and columns based on sorted indices
cm_sorted = cm[sorted_indices, :][:, sorted_indices]

# Normalize confusion matrix by the sum of rows
cm_normalized = cm_sorted.astype('float') / cm_sorted.sum(axis=1)[:, np.newaxis]

# Ensure no division by zero
cm_normalized[np.isnan(cm_normalized)] = 0

# Compute confusion matrix for calculating accuracies
cm_full = confusion_matrix(labels, all_preds.argmax(1))

# Calculate the accuracy for each taxon
accuracies = np.diag(cm_full) / cm_full.sum(axis=1) * 100

# Reorder accuracies to match the sorted taxon names
reordered_accuracies = accuracies[sorted_indices]

# Function to italicize taxon names except "var." and "subsp."
def italicize_name(name):
    words = name.split()
    italicized_words = [f"$\\it{{{word}}}$" if word not in ["var.", "subsp."] else word for word in words]
    return " ".join(italicized_words)

# Get the specimen counts sorted according to sorted_TaxonName
sorted_specimen_counts = [subfolder_counts[name] for name in sorted_TaxonName]

# Update taxon names with accuracy in parentheses for x-axis
updated_taxon_names_with_accuracy = [
    f"{italicize_name(name)} ({reordered_accuracies[idx]:.0f})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Update taxon names with specimen count in parentheses for y-axis
updated_taxon_names_with_counts = [
    f"{italicize_name(name)} ({sorted_specimen_counts[idx]})"
    for idx, name in enumerate(sorted_TaxonName)
]

# Plot the confusion matrix
plt.figure(figsize=(35, 27), dpi=300)
ax = sns.heatmap(cm_normalized, annot=False, fmt=".2f", vmax=1, xticklabels=updated_taxon_names_with_accuracy, yticklabels=updated_taxon_names_with_counts, cmap='cividis', linewidths=.0)
ax.set_xlabel('Predicted Taxon (%)', fontsize=32)
ax.set_ylabel('True Taxon (N)', fontsize=32)

# Adjust tick parameters
plt.tick_params(axis='x', which='major', labelsize=30)
plt.tick_params(axis='y', which='major', labelsize=30)

# Modify colorbar font size
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30)

# Italicize the tick labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', rotation_mode='anchor', fontsize=24)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=24)

ax.xaxis.set_tick_params(pad=10)

plt.show()